<a href="https://colab.research.google.com/github/sts-sadr/-Data-Science-A-Z-Real-Life-Data-Science-Exercises-Included-with-DSS/blob/master/Endpoint_layers_tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow --upgrade --quiet

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
tf.__version__

'2.0.0-dev20190912'

# Usage of endpoint layers in the Functional API

An "endpoint layer" has access to the model's targets, and creates arbitrary losses and metrics using `add_loss` and `add_metric`. This enables you to define losses and metrics that don't match the usual signature `fn(y_true, y_pred, sample_weight=None)`.

Note that you could have separate metrics for training and eval with this pattern.

In [0]:
class LogisticEndpoint(keras.layers.Layer):

  def __init__(self, name=None):
    super(LogisticEndpoint, self).__init__(name=name)
    self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

  def call(self, targets, logits, sample_weights=None):      
    # Compute the training-time loss value and add it
    # to the layer using `self.add_loss()`.
    loss = self.loss_fn(targets, logits, sample_weights)
    self.add_loss(loss)
    
    # Log the loss as a metric (we could log arbitrary metrics,
    # including different metrics for training and inference.
    self.add_metric(loss, name=self.name, aggregation='mean')

    # Return the inference-time prediction tensor (for `.predict()`).
    return tf.nn.softmax(logits)

In [0]:
inputs = keras.Input((764,), name='inputs')
logits = keras.layers.Dense(1)(inputs)
targets = keras.Input((1,), name='targets')
sample_weights = keras.Input((1,), name='sample_weights')
preds = LogisticEndpoint()(targets, logits, sample_weights)
model = keras.Model([inputs, targets, sample_weights], preds)

data = {
    'inputs': np.random.random((1000, 764)),
    'targets': np.random.random((1000, 1)),
    'sample_weights': np.random.random((1000, 1))
}

model.compile(keras.optimizers.Adam(1e-3))
model.fit(data, epochs=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 1s 572us/sample - loss: 0.3578 - logistic_endpoint: 0.3584
Epoch 2/2
1000/1000 [==============================] - 0s 61us/sample - loss: 0.3566 - logistic_endpoint: 0.3571


# Usage of loss endpoint layers in subclassed models

In [0]:
class LogReg(keras.Model):
  
  def __init__(self):
    super(LogReg, self).__init__()
    self.dense = keras.layers.Dense(1)
    self.logistic_endpoint = LogisticEndpoint()

  def call(self, inputs):
    # Note that all inputs should be in the first argument
    # since we want to be able to call `model.fit(inputs)`.
    logits = self.dense(inputs['inputs'])
    preds = self.logistic_endpoint(logits,
                                   inputs['targets'],
                                   inputs['sample_weights'])
    return preds
  

model = LogReg()
data = {
    'inputs': np.random.random((1000, 764)),
    'targets': np.random.random((1000, 1)),
    'sample_weights': np.random.random((1000, 1))
}

model.compile(keras.optimizers.Adam(1e-3))
model.fit(data, epochs=2)
    
    

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 409us/sample - loss: -0.9798 - logistic_endpoint_1: -1.0056
Epoch 2/2
1000/1000 [==============================] - 0s 63us/sample - loss: -4.0749 - logistic_endpoint_1: -4.1816


In [0]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)